In [ ]:
import json
import pandas as pd
import numpy as np


In [2]:
import json
import pandas as pd

# Carrega o arquivo JSON
with open('data/raw/revendedores_combustivel_20250802_170416.json', encoding='utf-8') as f:
    data = json.load(f)

# Extrai a lista de registros
registros = data["data"]

#1. DataFrame de informações cadastrais
colunas_cadastrais = [
    'codigoSIMP', 'autorizacao', 'dataPublicacao', 'razaoSocial', 'cnpj',
    'endereco', 'complemento', 'bairro', 'cep', 'uf', 'municipio',
    'distribuidora', 'dataVinculacao', 'classe'
]

# Normaliza apenas os campos desejados
dados_cadastrais = pd.json_normalize(registros)[colunas_cadastrais]

#  2. DataFrame de produtos 
dados_produtos = pd.json_normalize(
    registros,
    record_path='produtos',
    meta=['codigoSIMP']
)

#  3. DataFrame de localização 
dados_geo = pd.json_normalize(registros)[[
    'codigoSIMP',
    'latitude', 'longitude', 'latitude_ANP4C', 'longitude_ANP4C',
    'validacao', 'estimativaAcuracia', 'srid', 'src',
    'dataObtencao', 'origemInformacao', 'situacaoConstatada',
    'observacao', 'statusSIGAF'
]]


In [3]:
dados_cadastrais.columns = dados_cadastrais.columns.str.lower()
dados_produtos.columns = dados_produtos.columns.str.lower()
dados_geo.columns = dados_geo.columns.str.lower()

In [4]:
# mudando para maiúsculas as informações de localização
dados_geo = dados_geo.applymap(lambda x: x.upper() if isinstance(x, str) else x)

C:\Users\cleto\AppData\Local\Temp\ipykernel_18792\2758304496.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dados_geo = dados_geo.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [5]:
# Convertendo a coluna 'tancagem' para inteiro
dados_produtos['tancagem'] = dados_produtos['tancagem'].astype(int)

In [6]:
# colunas que possui informações em código ou pouco relevantes
dados_geo = dados_geo.drop(columns=['origeminformacao', 'observacao', 'situacaoconstatada', 'srid'])

In [ ]:
# da pra colocar NA aqui ou NaN 
dados_cadastrais.replace('', np.nan, inplace=True)
dados_produtos.replace('', np.nan, inplace=True)
dados_geo.replace('', np.nan, inplace=True)

In [ ]:
######### verificações ##############

In [79]:
# Verifica colunas com valores vazios (NaN ou string vazia) em dados_cadastrais
colunas_vazias = [col for col in dados_geo.columns if dados_geo[col].isnull().any() or (dados_geo[col] == '').any()]
print(colunas_vazias)

['latitude', 'longitude', 'latitude_anp4c', 'longitude_anp4c', 'estimativaacuracia', 'srid', 'dataobtencao', 'origeminformacao', 'situacaoconstatada', 'observacao', 'statussigaf']


In [108]:
dados_cadastrais

,codigosimp,autorizacao,datapublicacao,razaosocial,cnpj,endereco,complemento,bairro,cep,uf,municipio,distribuidora,datavinculacao,classe
0,1307222,PR/GO0248840,23/01/2025,POSTO STOCK MINEIROS LTDA,57649859000143,"RUA DA CURTICA, SN",QUADRA04 LOTE 02,LOTEAMENTO CAMBAUVA,75830163,GO,MINEIROS,BANDEIRA BRANCA,23/01/2025,
1,1307214,PR/RN0248841,23/01/2025,JOAO PAULO DA SILVA LOPES,52439755000190,"RUA JOAQUIM TORQUATO, 510",,CENTRO,59900000,RN,PAU DOS FERROS,BANDEIRA BRANCA,23/01/2025,
2,1307216,PR/MG0248842,23/01/2025,AUTO POSTO 3D DE BOM JESUS LTDA,57334901000137,"RODOVIA BR 116, SN",KM 726-200 METROS,BOM JESUS DA CACHOEIRA,36892971,MG,MURIAE,IPIRANGA,23/01/2025,
3,1307223,PR/GO0248843,23/01/2025,POSTO DE COMBUSTIVEL ABASTECE CALDAS NOVAS LTDA,47819366000150,"RUA B, S/N",QUADRA46 LOTE 01R AV S. FERN...,NOVA VILA,75681680,GO,CALDAS NOVAS,VIBRA,23/01/2025,
4,1307259,PR/SP0248844,24/01/2025,GDO PARTICIPACOES S/A,27751075016245,"RUA CANARIO, 674",,MOEMA,4521003,SP,SAO PAULO,IPIRANGA,24/01/2025,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,1025424,PR/CE0016859,14/11/2001,JOSE MARIA DE ARAUJO COMBUSTIVEIS,72470255000129,"ESTRADA DE GBA DO NORTE A CROATA, S/N",,BETÂNIA,62390000,CE,CROATA,BANDEIRA BRANCA,14/11/2001,
1228,1010854,PR/CE0018965,08/11/2001,POSTO SAGA EIRELI,04146342000175,"RODOVIA CE 138, SN",TERREODIST 01 KM DA SEDE,ZONA RURAL,63470000,CE,ERERE,BANDEIRA BRANCA,07/08/2020,
1229,1063399,PR/CE0224785,05/03/2008,SAMPAIO E LOPES LTDA,09316208000199,"ESTRADA DO FIO KM 04, 4040",,COACU,61760000,CE,EUSEBIO,RAIZEN,18/12/2017,
1230,1060004,PR/CE0218571,27/09/2007,GONCALVES COMERCIO DE COMBUSTVEIS LTDA.,08852620000160,"RUA CORONEL JOSE DANTAS, 613",,CENTRO,63200000,CE,MISSAO VELHA,VIBRA,19/03/2014,
